In [1]:
from pathlib import Path
import numpy as np
import pandas as pd # type: ignore


In [2]:
INPUT_PATH = Path('../../output/loans_call_report_recon.xlsx')
df = pd.read_excel(INPUT_PATH)

In [3]:
df['mjaccttypcd'].unique()

array(['BKCK', 'CK', 'CML', 'TD', 'SAV', 'MTG', 'MLN', 'CNS', 'LEAS',
       'RTMT'], dtype=object)

In [4]:
df = df[df['mjaccttypcd'].isin(['CML','MLN','MTG','CNS'])]

In [5]:
original_recon_amt = df['Net Balance'].sum()

This is our reconciliation amount exactly (total loan net Balance)
- From here, you can subtract out tax exempt bonds and get to the GL balance

In [6]:
df[df['fdiccatcd'].isnull()]

,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,bookbalance,...,rcf,availbalamt,fdiccatdesc,origbal,datemat,Net Balance,Net Available,Net Collateral Reserve,Total Exposure,orig_ttl_loan_amt
85,150289546,GAMMA REALTY LLC,Tax Exempt Bonds,9000000.00,VAR,CML,CM45,ACT,0.0431,6332682.65,...,5YR,0.0,NaN,9000000.0,2039-02-28,6332682.65,0.0,0.0,6332682.65,9000000.00
193,150314153,MOUNT SAINT CHARLES ACADEMY INC,Tax Exempt Bonds,0.01,VAR,CML,CM45,ACT,0.0710,3118692.40,...,5YR,0.0,NaN,0.0,2040-05-01,3118692.40,0.0,0.0,3118692.40,0.01
358,6253501,THE ARC OF BRISTOL COUNTY INC,Tax Exempt Bonds,0.00,VAR,CML,CM45,ACT,0.0500,1789910.19,...,5YR,0.0,NaN,0.0,2043-02-28,1789910.19,0.0,0.0,1789910.19,0.00
370,6252141,ASSOCIATES FOR HUMAN SERVICES INC,Tax Exempt Bonds,2900000.00,VAR,CML,CM45,ACT,0.0350,1720884.09,...,ANNU,0.0,NaN,2900000.0,2042-09-09,1720884.09,0.0,0.0,1720884.09,2900000.00
793,150888083,G LOPES CONSTRUCTION INC,Construction Master Line,0.00,FIX,MLN,ML02,ACT,0.0000,0.00,...,NaN,809901.0,NaN,NaN,2099-12-31,0.00,809901.0,0.0,809901.00,1250000.00
6110,150210840,PROPANE PLUS CORPORATION,Equipment Master Line,0.00,VAR,MLN,ML01,ACT,0.0400,0.00,...,NaN,150000.0,NaN,NaN,2099-12-31,0.00,150000.0,0.0,150000.00,150000.00


In [7]:
def cleaning_call_codes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleaning Stage for fdiccatcd
    - CML indirect get reclassified to AUTO
    - HOA gets its own category HOA
    - Tax Exempt Bonds become OTAL (other)
    - MTG loans are given their own code 'MTG', just for grouping purposes
    - Indirect Consumer loans originated by bank are put in Consumer/Other (CNOT)
    - Other/CML is the catch all for loans that don't have an FDIC code
    """
    df['fdiccatcd'] = np.where(df['currmiaccttypcd'].isin(['CM15','CM16']), 'AUTO', df['fdiccatcd'])
    df['fdiccatcd'] = np.where(df['currmiaccttypcd'].isin(['CM46','CM47']), 'HOA', df['fdiccatcd'])
    df['fdiccatcd'] = np.where(df['currmiaccttypcd'].isin(['CM45']), 'OTAL', df['fdiccatcd'])
    df['fdiccatcd'] = np.where(df['mjaccttypcd'].isin(['MTG']), 'MTG', df['fdiccatcd'])
    df['fdiccatcd'] = np.where(df['currmiaccttypcd'].isin(['IL09','IL10']), 'CNOT', df['fdiccatcd'])
    df['fdiccatcd'] = np.where(df['fdiccatcd'].isnull(), 'OTAL', df['fdiccatcd'])
    return df

In [8]:
df = cleaning_call_codes(df)

In [ ]:
fdic_groups = {
    # Note call codes have been adjusted in an earlier stage to stratify the portfolio
    'CRE': ['CNFM','OTCN','LAND','LNDV','RECN','REFI','REOE','REJU','REOW','RENO','REMU','OTAL','AGPR','REFM'],
    'C&I': ['CIUS'],
    'HOA': ['HOA'],
    'Residential': ['MTG'],
    'Consumer': ['CNOT','CNCR'],
    'Indirect': ['AUTO']
}
call_code_mapping = {code: group for group, codes in fdic_groups.items() for code in codes}
df['Category'] = df['fdiccatcd'].map(call_code_mapping)

final_df = df.groupby('Category')['Net Balance'].sum().reset_index()


In [12]:
final_df

,Category,Net Balance
0,C&I,1.166215e+08
1,CRE,1.112945e+09
2,Consumer,7.474293e+06
3,HOA,1.712787e+07
4,Indirect,2.805174e+08
5,Residential,9.210364e+08


In [13]:
original_recon_amt = round(original_recon_amt,2)

In [14]:
round(final_df['Net Balance'].sum(),2)

np.float64(2455722351.15)

In [15]:
original_recon_amt

np.float64(2455722351.15)

In [16]:
cml_slice = final_df[final_df['Category'].isin(['CRE','C&I','HOA'])]

In [17]:
cml_slice['Net Balance'].sum()

np.float64(1246694217.1)

This balances exactly

In [18]:
assert original_recon_amt - round(final_df['Net Balance'].sum()) < abs(1), "Failed Reconciliation"


In [19]:
final_df['Net Balance Rounded'] = (final_df['Net Balance'] / 1000).round()

In [20]:
final_df

,Category,Net Balance,Net Balance Rounded
0,C&I,1.166215e+08,116622.0
1,CRE,1.112945e+09,1112945.0
2,Consumer,7.474293e+06,7474.0
3,HOA,1.712787e+07,17128.0
4,Indirect,2.805174e+08,280517.0
5,Residential,9.210364e+08,921036.0
